In [ ]:
import robust_regression.sweeps.alpha_sweeps as alsw
import matplotlib.pyplot as plt
from robust_regression.fixed_point_equations.fpe_L2_loss import (
    var_hat_func_L2_decorrelated_noise,
)
from robust_regression.fixed_point_equations.fpe_BO import var_func_BO, var_hat_func_BO_num_decorrelated_noise
from robust_regression.fixed_point_equations.fpe_L1_loss import (
    var_hat_func_L1_decorrelated_noise,
)
from robust_regression.fixed_point_equations.fpe_Huber_loss import (
    var_hat_func_Huber_decorrelated_noise,
)
from robust_regression.fixed_point_equations.fpe_L2_regularization import var_func_L2
from robust_regression.aux_functions.misc import estimation_error, excess_gen_error, gen_error_BO, angle_teacher_student
import numpy as np

In [ ]:
def sigma_order_param(m, q, sigma):
    return sigma


def q_order_param(m, q, sigma):
    return q


def m_order_param(m, q, sigma):
    return m

In [ ]:
delta_in, delta_out, percentage, beta = 1.0, 0.5, 0.6, 0.2
alpha_min, alpha_max, n_alpha_pts = 0.01, 100000, 200
reg_param = 1e-3
a_hub = 1.0
delta_eff = (1 - percentage) * delta_in + percentage * delta_out
plateau_alpha_inf = (1 - percentage) * percentage**2 * (1 - beta) ** 2 + percentage * (1 - percentage) ** 2 * (
    beta - 1
) ** 2

In [ ]:
(
    alphas_L2,
    (gen_error_L2, sigmas_L2, qs_L2, ms_L2),
) = alsw.sweep_alpha_fixed_point(
    var_func_L2,
    var_hat_func_L2_decorrelated_noise,
    alpha_min,
    alpha_max,
    n_alpha_pts,
    {"reg_param": reg_param},
    {
        "delta_in": delta_in,
        "delta_out": delta_out,
        "percentage": percentage,
        "beta": beta,
    },
    initial_cond_fpe=(0.6, 0.01, 0.9),
    funs=[excess_gen_error, sigma_order_param, q_order_param, m_order_param],
    funs_args=[{"delta_in": delta_in, "delta_out": delta_out, "percentage": percentage, "beta": beta}, {}, {}, {}],
)

print("L2 done")

(
    alphas_L1,
    (gen_error_L1, sigmas_L1, qs_L1, ms_L1),
) = alsw.sweep_alpha_fixed_point(
    var_func_L2,
    var_hat_func_L1_decorrelated_noise,
    alpha_min,
    alpha_max,
    n_alpha_pts,
    {"reg_param": reg_param},
    {
        "delta_in": delta_in,
        "delta_out": delta_out,
        "percentage": percentage,
        "beta": beta,
    },
    initial_cond_fpe=(0.6, 0.01, 0.9),
    funs=[excess_gen_error, sigma_order_param, q_order_param, m_order_param],
    funs_args=[{"delta_in": delta_in, "delta_out": delta_out, "percentage": percentage, "beta": beta}, {}, {}, {}],
)

print("L1 done")

(
    alphas_Hub,
    (gen_error_Hub, sigmas_Hub, qs_Hub, ms_Hub),
) = alsw.sweep_alpha_fixed_point(
    var_func_L2,
    var_hat_func_Huber_decorrelated_noise,
    alpha_min,
    alpha_max,
    n_alpha_pts,
    {"reg_param": reg_param},
    {"delta_in": delta_in, "delta_out": delta_out, "percentage": percentage, "beta": beta, "a": a_hub},
    initial_cond_fpe=(0.6, 0.01, 0.9),
    funs=[excess_gen_error, sigma_order_param, q_order_param, m_order_param],
    funs_args=[{"delta_in": delta_in, "delta_out": delta_out, "percentage": percentage, "beta": beta}, {}, {}, {}],
)

print("Huber done")

In [ ]:
plt.figure(figsize=(7, 7))

plt.title(
    "$\\alpha$ sweep, $\\Delta_{{in}} = {}$, $\\Delta_{{out}} = {}$, $\\epsilon = {}$, $\\beta = {}$, $\\lambda = {}$, $a = {}$".format(
        delta_in, delta_out, percentage, beta, reg_param, a_hub
    )
)

plt.plot(alphas_L2, np.arccos(ms_L2 / np.sqrt(qs_L2)) / np.pi, label="L2")
plt.plot(alphas_L1, np.arccos(ms_L1 / np.sqrt(qs_L1)) / np.pi, label="L1")
plt.plot(alphas_Hub, np.arccos(ms_Hub / np.sqrt(qs_Hub)) / np.pi, label="Huber")

plt.yscale("log")
plt.xscale("log")
plt.ylabel(r"$\theta$")
plt.xlabel(r"$\alpha$")
plt.legend()
plt.grid()

plt.show()